#### PART 1 : Import necessary functions, set parameters and load your own recordings ####

**Import all the librairies and functions**

In [1]:
# Importation of librairies and functions
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

In [4]:
# check some package versions for documentation and reproducibility
import sys
import mne
from matplotlib import __version__ as plt_version
import scipy
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('matplotlib', plt_version)

Python sys 3.10.9 | packaged by conda-forge | (main, Feb  2 2023, 20:14:58) [MSC v.1929 64 bit (AMD64)]
pandas 1.5.3
numpy 1.23.5
mne 1.3.0
sci-py 1.10.0
matplotlib 3.6.3


In [5]:
def set_cd_repo_folder():
    """sets current working directory to main repo folder"""
    cd = os.getcwd()

    check = 0

    while os.path.basename(cd) != 'ReSync':

        cd = os.path.dirname(cd)
        check += 1
        if check > 10: raise ValueError('Repo path not found')
    
    os.chdir(cd)

    print(f'working directory changed to {os.getcwd()}')

    return os.getcwd()


In [6]:
project_path = set_cd_repo_folder()

working directory changed to c:\Users\Juliette\Research\Projects\Synchronization_project\Code\ReSync


In [7]:
# import custom-made functions
import functions.preprocessing as preproc
import functions.utils as utils
import functions.plotting as plot
import functions.find_artefacts as artefact
import functions.crop as crop
import functions.main_resync as resync
#import functions.plotting_interactive as plot_interact

In [8]:
importlib.reload(plot)
importlib.reload(preproc)
importlib.reload(utils)
importlib.reload(artefact)
importlib.reload(crop)
importlib.reload(resync)
#importlib.reload(plot_interact)

<module 'functions.plotting_interactive' from 'c:\\Users\\Juliette\\Research\\Projects\\Synchronization_project\\Code\\ReSync\\functions\\plotting_interactive.py'>

**Load your own LFP data:**

Resulting variables needed for subsequent analysis:
- LFP_array (np.ndarray, multi dimensional): the LFP recording which has to be aligned, containing all channels
- lfp_sig (np.ndarray, 1d): the channel containing the LFP signal from the hemisphere where the stimulation was delivered to create artefacts
- LFP_rec_ch_names (list): names of all the channels, in a list (will be used to annotate cropped recording)


In [9]:
# load pyPerceive functions
os.chdir(os.path.dirname(os.getcwd()))
os.chdir(os.path.join(os.getcwd(), 'PyPerceive'))
os.chdir(os.path.join(os.getcwd(), 'code'))
pyPerceive_path = os.getcwd()
print (f'working dir to go fetch PyPerceive functions:{pyPerceive_path}')

from PerceiveImport.classes import (
    main_class, modality_class, metadata_class,
    session_class, condition_class, task_class,
    contact_class, run_class
)
import PerceiveImport.methods.load_rawfile as load_rawfile
import PerceiveImport.methods.find_folders as find_folders
import PerceiveImport.methods.metadata_helpers as metaHelpers

#reset the proper working directory for the analysis
os.chdir(project_path)
print (f'working dir set back to:{project_path}')

working dir to go fetch PyPerceive functions:c:\Users\Juliette\Research\Projects\Synchronization_project\Code\PyPerceive\code
working dir set back to:c:\Users\Juliette\Research\Projects\Synchronization_project\Code\ReSync


In [10]:
# choose LFP file
sub041 = main_class.PerceiveData(
    sub = "041", 
    incl_modalities=['streaming'],
    incl_session = ["fu18m"],
    incl_condition =['m1s0','m1s1'],
    incl_task = ["rest"],
    # incl_contact = ["RingL", "SegmInterR", "SegmIntraR"],
    import_json=False,
    warn_for_metaNaNs=True,
    allow_NaNs_in_metadata=False
)

# define LFP data
LFP_rec = sub041.streaming.fu18m.m1s0.rest.run1.data
LFP_array = LFP_rec.get_data()
ch_i = 0 #choose index of the channel containing the stim artefacts (O for left hemisphere, 1 for right hemisphere)
lfp_sig = LFP_rec.get_data()[ch_i]
LFP_rec_ch_names = LFP_rec.ch_names

n_chan = len(LFP_rec.ch_names)
time_duration_LFP = (LFP_rec.n_times/LFP_rec.info['sfreq']).astype(float)
print(     
	f'The data object has:\n\t{LFP_rec.n_times} time samples,'      
	f'\n\tand a sample frequency of {LFP_rec.info["sfreq"]} Hz'      
	f'\n\twith a recording duration of {time_duration_LFP} seconds.'      
	f'\n\t{n_chan} channels were labeled as \n{LFP_rec.ch_names}.')
print(f'The channel containing artefacts has index {ch_i} and is named {LFP_rec.ch_names[ch_i]}')


	### WARNING: NaNs in Metadata Table sub-041 ###
NaNs in: sub-20220404PStn_ses-2022071206564297_run-BrainSense20220712073400.mat
NaNs in: sub-20220404PStn_ses-2022071206564297_run-BrainSense20220712075100.mat
NaNs in: sub-20220404PStn_ses-2022071206564297_run-BrainSense20220712080900.mat
NaNs in: sub-20220404PStn_ses-2022071206564297_run-BrainSense20220712082700.mat
NaNs in: sub-20220404PStn_ses-2022071206564297_run-BrainSense20220712084400.mat
add run 1
Creating RawArray with float64 data, n_channels=6, n_times=69063
    Range : 0 ... 69062 =      0.000 ...   276.248 secs
Ready.
add run 1
Creating RawArray with float64 data, n_channels=6, n_times=78937
    Range : 0 ... 78936 =      0.000 ...   315.744 secs
Ready.
add run 2
Creating RawArray with float64 data, n_channels=6, n_times=107625
    Range : 0 ... 107624 =      0.000 ...   430.496 secs
Ready.
add run 3
Creating RawArray with float64 data, n_channels=6, n_times=27938
    Range : 0 ... 27937 =      0.000 ...   111.748 secs
Rea

**Load your own external data:**
(our external data recorder is a TMSi Data recorder.)

PM: NOTICE THE POP UP WINDOW AFTER RUNNING, TO SELECT THE FILE LOCATION

Resulting variables:
- external_file (np.ndarray, multi-dimensional): the complete external recording containing all channels recorded
- BIP_channel (np.ndarray, 1d): the channel containing the signal from the bipolar electrode used to pick up the artefacts on the IPG/cable
- external_rec_ch_names (list, same length as the number of channels in external_file): list of the channels names, to rename them accordingly after alignment

In [11]:
import functions.tmsi_poly5reader as poly5_reader
import functions.loading_TMSi as loading

In [12]:
TMSi_data = poly5_reader.Poly5Reader()  # open TMSi data from poly5
(BIP_channel,
 external_file,
 external_rec_ch_names) = loading.load_TMSi_artefact_channel(TMSi_data) # function adapted for our own data recorder, 
# to load all necessary variables

Reading file  C:/Users/Juliette/OneDrive - Charité - Universitätsmedizin Berlin/Recordings/TMSi files/sub-041/sub_041_18MFU_M1S0_BrStr_restTap - 20230907T134210/sub_041_18MFU_M1S0_BrStr_restTap-20230907T134210.DATA.Poly5
	 Number of samples:  1276704 
	 Number of channels:  12 
	 Sample rate: 4096 Hz
Done reading data.
Creating RawArray with float64 data, n_channels=12, n_times=1276704
    Range : 0 ... 1276703 =      0.000 ...   311.695 secs
Ready.
The data object has:
	1276704 time samples,
	and a sample frequency of 4096.0 Hz
	with a recording duration of 311.6953125 seconds.
	12 channels were labeled as 
['BIP 01', 'BIP 02', 'BIP 03', 'BIP 04', 'X-0', 'Y-0', 'Z-0', 'X-1', 'Y-1', 'Z-1', 'STATUS', 'Counter 2power24'].
the channel used to align datas is the channel named BIP 01 and has index 0


**READ ME**

Before starting the run_resync function, be careful to check that the config file is properly set. In particular, pay attention to:
- write the proper subject ID (to not overwrite previous analysis)
- write the correct sampling frequencies corresponding to YOUR recordings
- by default use kernel "1", and set "real_index_LFP" to 0 for the first run. This can be adjusted if necessary before re-running.
- by default, set "consider_first_seconds_LFP", "consider_first_seconds_external" and "ignore_first_seconds_external" to null. 

#### PART 2: Align recordings: ####

In [13]:
(LFP_df_offset, 
 external_df_offset) = resync.run_resync(
    LFP_array=LFP_array,
    lfp_sig=lfp_sig,
    LFP_rec_ch_names=LFP_rec_ch_names,
    external_file=external_file,
    BIP_channel=BIP_channel,
    external_rec_ch_names=external_rec_ch_names,
    SHOW_FIGURES = True
)

-6.712695015282861e-11
Alignment performed ! 
Please check carefully in all figures that the samples selected 
as start of the artefact are correct, and if they are not 
please correct parameters accordingly in the config file before re-running


#### PART 3 : Look for timeshift ####

In [ ]:
resync.run_timeshift_analysis(
    LFP_df_offset,
    external_df_offset,
    SHOW_FIGURES = True
)
